# read

In [1]:
import pandas as pd; pd.set_option('display.max_rows', None)
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('./data/train_df.csv', encoding='cp949')
test  = pd.read_csv('./data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('./data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


In [4]:
test.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,32000,5013000043028400,1165000021097200,식품,농산물
1,32001,5013000044016100,1154500002066400,식품,농산물
2,32002,5013000205030200,4139000102013200,식품,농산물
3,32003,5013000205030200,4221000040093400,식품,농산물
4,32004,5013000268011400,2726000004017100,식품,농산물


# preprocessing

In [5]:
train['ag'] = 0
test['ag']  = 0

train.loc[train['DL_GD_MCLS_NM'] == '농산물', 'ag'] = 1
test.loc[test['DL_GD_MCLS_NM'] == '농산물', 'ag'] = 1

In [6]:
large_label = dict(train.groupby('DL_GD_LCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
mid_label   = dict(train.groupby('DL_GD_MCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)

train['DL_GD_LCLS_NM'] = train['DL_GD_LCLS_NM'].map(large_label)
test['DL_GD_LCLS_NM']  = test['DL_GD_LCLS_NM'].map(large_label)

train['DL_GD_MCLS_NM'] = train['DL_GD_MCLS_NM'].map(mid_label)
test['DL_GD_MCLS_NM']  = test['DL_GD_MCLS_NM'].map(mid_label)

In [7]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,0
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,0
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,0
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,1
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,0


In [8]:
# 격자공간고유번호 슬라이싱

for i in [3, 4, 5, 6, 7, 8, 9]:
    train['send_' + str(i)] = train['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['send_' + str(i)]  = test['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    
    train['rec_' + str(i)] = train['REC_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['rec_' + str(i)]  = test['REC_SPG_INNB'].astype('str').str[:i].astype('int64')

In [9]:
train['eq'] = 0
test['eq']  = 0

train.loc[train['SEND_SPG_INNB'] == train['REC_SPG_INNB'], 'eq'] = 1
test.loc[test['SEND_SPG_INNB'] == test['REC_SPG_INNB'], 'eq'] = 1

In [10]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag,send_3,rec_3,send_4,...,rec_5,send_6,rec_6,send_7,rec_7,send_8,rec_8,send_9,rec_9,eq
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,0,112,501,1129,...,50110,112900,501100,1129000,5011000,11290000,50110002,112900001,501100022,0
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,0,113,501,1135,...,50110,113500,501100,1135000,5011000,11350000,50110001,113500000,501100017,0
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,0,113,501,1135,...,50110,113500,501100,1135000,5011000,11350000,50110002,113500003,501100026,0
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,1,115,501,1154,...,50110,115450,501100,1154500,5011000,11545000,50110003,115450000,501100031,0
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,0,116,501,1165,...,50110,116500,501100,1165000,5011000,11650000,50110001,116500002,501100017,0


# encoding

In [11]:
train.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)
test.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)

In [12]:
train.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag,send_3,rec_3,send_4,rec_4,send_5,rec_5,send_6,rec_6,send_7,rec_7,send_8,rec_8,send_9,rec_9,eq
0,6.678694,3.672897,3,0,112,501,1129,5011,11290,50110,112900,501100,1129000,5011000,11290000,50110002,112900001,501100022,0
1,5.407921,4.444134,3,0,113,501,1135,5011,11350,50110,113500,501100,1135000,5011000,11350000,50110001,113500000,501100017,0
2,6.678694,10.375479,9,0,113,501,1135,5011,11350,50110,113500,501100,1135000,5011000,11350000,50110002,113500003,501100026,0
3,4.658195,4.297401,10,1,115,501,1154,5011,11545,50110,115450,501100,1154500,5011000,11545000,50110003,115450000,501100031,0
4,4.658195,6.807151,3,0,116,501,1165,5011,11650,50110,116500,501100,1165000,5011000,11650000,50110001,116500002,501100017,0


# modeling

In [13]:
X = train.drop('INVC_CONT', axis=1)
y = train['INVC_CONT']

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(random_state=0, random_strength=0)

model.fit(X, y)
pd.DataFrame({'col':X.columns, 'fi':model.feature_importances_})

In [14]:
from sklearn.model_selection import GridSearchCV

model = CatBoostRegressor(random_state=0, random_strength=0)

param = {
    
}

gscv = GridSearchCV(
    estimator=model, param_grid=param, scoring='neg_root_mean_squared_error', 
    refit=True, cv=5
)

gscv.fit(X, y)

Learning rate set to 0.068341
0:	learn: 5.4330044	total: 182ms	remaining: 3m 1s
1:	learn: 5.4201316	total: 205ms	remaining: 1m 42s
2:	learn: 5.4094266	total: 231ms	remaining: 1m 16s
3:	learn: 5.3841307	total: 262ms	remaining: 1m 5s
4:	learn: 5.3753344	total: 292ms	remaining: 58.2s
5:	learn: 5.3673881	total: 324ms	remaining: 53.6s
6:	learn: 5.3608260	total: 347ms	remaining: 49.2s
7:	learn: 5.3539767	total: 391ms	remaining: 48.5s
8:	learn: 5.3331565	total: 443ms	remaining: 48.7s
9:	learn: 5.3278950	total: 479ms	remaining: 47.5s
10:	learn: 5.3095222	total: 513ms	remaining: 46.1s
11:	learn: 5.3051231	total: 538ms	remaining: 44.3s
12:	learn: 5.3010735	total: 623ms	remaining: 47.3s
13:	learn: 5.2862129	total: 667ms	remaining: 47s
14:	learn: 5.2822180	total: 688ms	remaining: 45.2s
15:	learn: 5.2673446	total: 717ms	remaining: 44.1s
16:	learn: 5.2541951	total: 744ms	remaining: 43s
17:	learn: 5.2423019	total: 771ms	remaining: 42.1s
18:	learn: 5.2391733	total: 853ms	remaining: 44s
19:	learn: 5.22

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x0000026F59526310>,
             param_grid={}, scoring='neg_root_mean_squared_error')

In [15]:
print(gscv.best_params_)
print(-gscv.best_score_)

{}
5.625949853472752


In [16]:
pred = gscv.predict(test)
sub['INVC_CONT'] = pred

In [17]:
sub.head()

,index,INVC_CONT
0,32000,4.528327
1,32001,4.738477
2,32002,5.040101
3,32003,5.868895
4,32004,4.840474


In [18]:
# sub.to_csv('./sub/logistics_06.csv', index=False)